In [1]:
!pip install transformers -q
!pip install sentencepiece -q
!pip install datasets -q

     |████████████████████████████████| 3.1 MB 4.2 MB/s 
     |████████████████████████████████| 596 kB 64.7 MB/s 
     |████████████████████████████████| 59 kB 7.8 MB/s 
     |████████████████████████████████| 895 kB 58.4 MB/s 
     |████████████████████████████████| 3.3 MB 44.0 MB/s 
     |████████████████████████████████| 1.2 MB 4.3 MB/s 
     |████████████████████████████████| 298 kB 4.0 MB/s 
     |████████████████████████████████| 1.1 MB 55.5 MB/s 
     |████████████████████████████████| 132 kB 72.5 MB/s 
     |████████████████████████████████| 243 kB 68.8 MB/s 
     |████████████████████████████████| 160 kB 65.3 MB/s 
     |████████████████████████████████| 271 kB 52.9 MB/s 
     |████████████████████████████████| 192 kB 61.5 MB/s 


In [2]:
from transformers import AutoModel, AutoTokenizer, AutoModelForSequenceClassification, AutoConfig
import pandas as pd
import numpy as np
from transformers import Trainer, TrainingArguments
from sklearn.model_selection import train_test_split
from datasets import DatasetDict
from datasets import load_metric
import torch

In [3]:
from IPython.display import display, HTML

In [4]:
import warnings
warnings.filterwarnings('ignore')

In [5]:
class LyricDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

In [10]:
def get_train_test_sets(data,tokenizer,label='iGenre',feature='Lyric'):
  tmp_tr, tmp_te = train_test_split(data[label])
  train_set = pd.DataFrame()
  train_set[feature] = [data.loc[x,feature] for x in tmp_tr.index]
  train_set[label] = [data.loc[x,label] for x in tmp_tr.index]

  test_set = pd.DataFrame()
  test_set[feature] = [data.loc[x,feature] for x in tmp_te.index]
  test_set[label] = [data.loc[x,label] for x in tmp_te.index]
  train_encodings = tokenizer(list(train_set[feature]), truncation=True, padding=True, max_length=96)
  test_encodings = tokenizer(list(test_set[feature]), truncation=True, padding=True, max_length=96)
  
  return LyricDataset(train_encodings, list(train_set[label])), LyricDataset(test_encodings, list(test_set[label]))

In [7]:
def compute_metrics(eval_pred):
    f1 = load_metric('f1')
    pre = load_metric('precision')
    rec = load_metric('recall')
    predictions, labels = eval_pred
    predictions = predictions.argmax(axis=-1)
    return {'f1':f1.compute(predictions=predictions, references=labels)['f1'],
            'precision':pre.compute(predictions=predictions, references=labels)['precision'],
            'recall':rec.compute(predictions=predictions, references=labels)['recall']
            }

In [9]:
data = pd.read_pickle('/content/drive/MyDrive/mp3_chroma.pkl')

In [11]:
def generate_trainer(data, chk_point, model_init, eval_steps=50, evaluation_strategy="steps",num_epochs=5):
    tokenizer=AutoTokenizer.from_pretrained(chk_point)
    train_set, test_set = get_train_test_sets(data, tokenizer)
    training_args = TrainingArguments("test", evaluation_strategy=evaluation_strategy, 
                                      eval_steps=eval_steps, disable_tqdm=True,
                                      logging_dir='./logs',
                                      per_device_train_batch_size=16,
                                      per_device_eval_batch_size=64,
                                      load_best_model_at_end=True
                                      )
    trainer = Trainer(
        args=training_args,
        train_dataset=train_set,
        eval_dataset=test_set,
        model_init=model_init,
        compute_metrics=compute_metrics,
    )
    return trainer, train_set, test_set

## Indic BERT Single Modal

In [18]:
def indic_bert_model_init():
    return AutoModelForSequenceClassification.from_pretrained('/content/drive/MyDrive/IndicBERT_FineTuned/pytorch_model.bin',
                                                              config=AutoConfig.from_pretrained('/content/drive/MyDrive/IndicBERT_FineTuned/config.json')
    )

In [19]:
trainer, _, _ = generate_trainer(data, 'ai4bharat/indic-bert', indic_bert_model_init, eval_steps=50)
train_metrics = trainer.train()

Could not locate the tokenizer configuration file, will try to use the model config instead.
loading configuration file https://huggingface.co/ai4bharat/indic-bert/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/2d290a1a22a5f80e173def8b2f31f12d68a957542e6769ab06bfc3de06bc49f4.06ba3893e888d6ff1388c45cdbee1fb785542ae22b70ff159f55da323230a159
Model config AlbertConfig {
  "attention_probs_dropout_prob": 0,
  "bos_token_id": 2,
  "classifier_dropout_prob": 0.1,
  "down_scale_factor": 1,
  "embedding_size": 128,
  "eos_token_id": 3,
  "gap_size": 0,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "inner_group_num": 1,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "albert",
  "net_structure_type": 0,
  "num_attention_heads": 12,
  "num_hidden_groups": 1,
  "num_hidden_layers": 12,
  "num_memory_blocks": 0,
  "pad_token_id": 0,
  "position_embedding

Downloading:   0%|          | 0.00/2.07k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.10k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.10k [00:00<?, ?B/s]

{'eval_loss': 0.3695986568927765, 'eval_f1': 0.900763358778626, 'eval_precision': 0.885, 'eval_recall': 0.917098445595855, 'eval_runtime': 4.8326, 'eval_samples_per_second': 58.975, 'eval_steps_per_second': 1.035, 'epoch': 0.93}


...

{'eval_loss': 0.34496110677719116, 'eval_f1': 0.9015544041450776, 'eval_precision': 0.9015544041450777, 'eval_recall': 0.9015544041450777, 'eval_runtime': 3.402, 'eval_samples_per_second': 83.773, 'eval_steps_per_second': 1.47, 'epoch': 1.85}


***** Running Evaluation *****
  Num examples = 285
  Batch size = 64


{'eval_loss': 0.3743305504322052, 'eval_f1': 0.8992248062015503, 'eval_precision': 0.8969072164948454, 'eval_recall': 0.9015544041450777, 'eval_runtime': 3.3508, 'eval_samples_per_second': 85.054, 'eval_steps_per_second': 1.492, 'epoch': 2.78}




Training completed. Do not forget to share your model on huggingface.co/models =)




{'train_runtime': 35.4103, 'train_samples_per_second': 72.352, 'train_steps_per_second': 4.575, 'train_loss': 0.3657610622453101, 'epoch': 3.0}


In [20]:
eval_metrics = trainer.evaluate()

***** Running Evaluation *****
  Num examples = 285
  Batch size = 64


{'eval_loss': 0.37395983934402466, 'eval_f1': 0.8969072164948454, 'eval_precision': 0.8923076923076924, 'eval_recall': 0.9015544041450777, 'eval_runtime': 3.3393, 'eval_samples_per_second': 85.348, 'eval_steps_per_second': 1.497, 'epoch': 3.0}


In [21]:
trainer.create_model_card()

Dropping the following result as it does not have all the necessary fields:
{'task': {'name': 'Text Classification', 'type': 'text-classification'}, 'metrics': [{'name': 'F1', 'type': 'f1', 'value': 0.8969072164948454}, {'name': 'Precision', 'type': 'precision', 'value': 0.8923076923076924}, {'name': 'Recall', 'type': 'recall', 'value': 0.9015544041450777}]}


## MURIL Single Modal

In [24]:
def muril_model_init():
    return AutoModelForSequenceClassification.from_pretrained('/content/drive/MyDrive/MURIL_FineTuned/pytorch_model.bin',
                                                              config=AutoConfig.from_pretrained('/content/drive/MyDrive/MURIL_FineTuned/config.json'))

In [25]:
trainer, _, _ = generate_trainer(data, 'google/muril-base-cased', muril_model_init, eval_steps=50)
train_metrics = trainer.train()

https://huggingface.co/google/muril-base-cased/resolve/main/tokenizer_config.json not found in cache or force_download set to True, downloading to /root/.cache/huggingface/transformers/tmpurgw52jt


Downloading:   0%|          | 0.00/181 [00:00<?, ?B/s]

storing https://huggingface.co/google/muril-base-cased/resolve/main/tokenizer_config.json in cache at /root/.cache/huggingface/transformers/677de489a9460181a44c530abce89d7d341485a3d49dacc89a9b5a1fed34c328.c685950e8d32cffa796a9b56861350d915f216970457f689f9fe9d9e90f06e24
creating metadata file for /root/.cache/huggingface/transformers/677de489a9460181a44c530abce89d7d341485a3d49dacc89a9b5a1fed34c328.c685950e8d32cffa796a9b56861350d915f216970457f689f9fe9d9e90f06e24
https://huggingface.co/google/muril-base-cased/resolve/main/config.json not found in cache or force_download set to True, downloading to /root/.cache/huggingface/transformers/tmp3qpysxyy


Downloading:   0%|          | 0.00/411 [00:00<?, ?B/s]

storing https://huggingface.co/google/muril-base-cased/resolve/main/config.json in cache at /root/.cache/huggingface/transformers/d8ca6ce642f067ecf3d1163f4d2903b471287613933f2857ca8307e500bc7645.aff1657f5771205f5a0c6cb4816f125ee5f2f2d62dbf27e6b9fee30b0ebbf0f5
creating metadata file for /root/.cache/huggingface/transformers/d8ca6ce642f067ecf3d1163f4d2903b471287613933f2857ca8307e500bc7645.aff1657f5771205f5a0c6cb4816f125ee5f2f2d62dbf27e6b9fee30b0ebbf0f5
loading configuration file https://huggingface.co/google/muril-base-cased/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/d8ca6ce642f067ecf3d1163f4d2903b471287613933f2857ca8307e500bc7645.aff1657f5771205f5a0c6cb4816f125ee5f2f2d62dbf27e6b9fee30b0ebbf0f5
Model config BertConfig {
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "embedding_size": 768,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range

...

{'eval_loss': 0.29967671632766724, 'eval_f1': 0.9255319148936171, 'eval_precision': 0.9508196721311475, 'eval_recall': 0.9015544041450777, 'eval_runtime': 3.3548, 'eval_samples_per_second': 84.953, 'eval_steps_per_second': 1.49, 'epoch': 1.85}


***** Running Evaluation *****
  Num examples = 285
  Batch size = 64


{'eval_loss': 0.33477863669395447, 'eval_f1': 0.9278350515463918, 'eval_precision': 0.9230769230769231, 'eval_recall': 0.9326424870466321, 'eval_runtime': 3.3814, 'eval_samples_per_second': 84.285, 'eval_steps_per_second': 1.479, 'epoch': 2.78}




Training completed. Do not forget to share your model on huggingface.co/models =)




{'train_runtime': 41.6057, 'train_samples_per_second': 61.578, 'train_steps_per_second': 3.894, 'train_loss': 0.3191578241042149, 'epoch': 3.0}


In [26]:
eval_metrics = trainer.evaluate()

***** Running Evaluation *****
  Num examples = 285
  Batch size = 64


{'eval_loss': 0.3465764820575714, 'eval_f1': 0.9254498714652956, 'eval_precision': 0.9183673469387755, 'eval_recall': 0.9326424870466321, 'eval_runtime': 3.3342, 'eval_samples_per_second': 85.478, 'eval_steps_per_second': 1.5, 'epoch': 3.0}


In [27]:
trainer.create_model_card()

Dropping the following result as it does not have all the necessary fields:
{'task': {'name': 'Text Classification', 'type': 'text-classification'}, 'metrics': [{'name': 'F1', 'type': 'f1', 'value': 0.9254498714652956}, {'name': 'Precision', 'type': 'precision', 'value': 0.9183673469387755}, {'name': 'Recall', 'type': 'recall', 'value': 0.9326424870466321}]}


## XLM-R Single Modal

In [28]:
def xlmr_model_init():
    return AutoModelForSequenceClassification.from_pretrained('/content/drive/MyDrive/XLMR_Finetuned/pytorch_model.bin',
                                                              config=AutoConfig.from_pretrained('/content/drive/MyDrive/XLMR_Finetuned/config.json'))

In [29]:
trainer, _, _ = generate_trainer(data, 'xlm-roberta-base', xlmr_model_init, eval_steps=50)
train_metrics = trainer.train()

Could not locate the tokenizer configuration file, will try to use the model config instead.
https://huggingface.co/xlm-roberta-base/resolve/main/config.json not found in cache or force_download set to True, downloading to /root/.cache/huggingface/transformers/tmpa6k6tyxb


Downloading:   0%|          | 0.00/512 [00:00<?, ?B/s]

storing https://huggingface.co/xlm-roberta-base/resolve/main/config.json in cache at /root/.cache/huggingface/transformers/87683eb92ea383b0475fecf99970e950a03c9ff5e51648d6eee56fb754612465.ab95cf27f9419a99cce4f19d09e655aba382a2bafe2fe26d0cc24c18cf1a1af6
creating metadata file for /root/.cache/huggingface/transformers/87683eb92ea383b0475fecf99970e950a03c9ff5e51648d6eee56fb754612465.ab95cf27f9419a99cce4f19d09e655aba382a2bafe2fe26d0cc24c18cf1a1af6
loading configuration file https://huggingface.co/xlm-roberta-base/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/87683eb92ea383b0475fecf99970e950a03c9ff5e51648d6eee56fb754612465.ab95cf27f9419a99cce4f19d09e655aba382a2bafe2fe26d0cc24c18cf1a1af6
Model config XLMRobertaConfig {
  "architectures": [
    "XLMRobertaForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "in

Downloading:   0%|          | 0.00/4.83M [00:00<?, ?B/s]

storing https://huggingface.co/xlm-roberta-base/resolve/main/sentencepiece.bpe.model in cache at /root/.cache/huggingface/transformers/9df9ae4442348b73950203b63d1b8ed2d18eba68921872aee0c3a9d05b9673c6.00628a9eeb8baf4080d44a0abe9fe8057893de20c7cb6e6423cddbf452f7d4d8
creating metadata file for /root/.cache/huggingface/transformers/9df9ae4442348b73950203b63d1b8ed2d18eba68921872aee0c3a9d05b9673c6.00628a9eeb8baf4080d44a0abe9fe8057893de20c7cb6e6423cddbf452f7d4d8
https://huggingface.co/xlm-roberta-base/resolve/main/tokenizer.json not found in cache or force_download set to True, downloading to /root/.cache/huggingface/transformers/tmp5rps8cpm


...

{'eval_loss': 0.48788216710090637, 'eval_f1': 0.9077669902912622, 'eval_precision': 0.8538812785388128, 'eval_recall': 0.9689119170984456, 'eval_runtime': 3.35, 'eval_samples_per_second': 85.075, 'eval_steps_per_second': 1.493, 'epoch': 1.85}


***** Running Evaluation *****
  Num examples = 285
  Batch size = 64


{'eval_loss': 0.3242870569229126, 'eval_f1': 0.9246231155778895, 'eval_precision': 0.8975609756097561, 'eval_recall': 0.9533678756476683, 'eval_runtime': 4.1547, 'eval_samples_per_second': 68.596, 'eval_steps_per_second': 1.203, 'epoch': 2.78}




Training completed. Do not forget to share your model on huggingface.co/models =)




{'train_runtime': 43.0154, 'train_samples_per_second': 59.56, 'train_steps_per_second': 3.766, 'train_loss': 0.2925580578085817, 'epoch': 3.0}


In [30]:
eval_metrics = trainer.evaluate()

***** Running Evaluation *****
  Num examples = 285
  Batch size = 64


{'eval_loss': 0.32089340686798096, 'eval_f1': 0.9265822784810126, 'eval_precision': 0.905940594059406, 'eval_recall': 0.9481865284974094, 'eval_runtime': 3.3203, 'eval_samples_per_second': 85.836, 'eval_steps_per_second': 1.506, 'epoch': 3.0}


In [31]:
trainer.create_model_card()

Dropping the following result as it does not have all the necessary fields:
{'task': {'name': 'Text Classification', 'type': 'text-classification'}, 'metrics': [{'name': 'F1', 'type': 'f1', 'value': 0.9265822784810126}, {'name': 'Precision', 'type': 'precision', 'value': 0.905940594059406}, {'name': 'Recall', 'type': 'recall', 'value': 0.9481865284974094}]}


## MBERT Single Modal

In [32]:
def mbert_model_init():
    return AutoModelForSequenceClassification.from_pretrained('/content/drive/MyDrive/MBERT_Finetuned/pytorch_model.bin',
                                                              config=AutoConfig.from_pretrained('/content/drive/MyDrive/MBERT_Finetuned/config.json'))

In [33]:
trainer, _, _ = generate_trainer(data, 'bert-base-multilingual-cased', mbert_model_init, eval_steps=50)
train_metrics = trainer.train()

https://huggingface.co/bert-base-multilingual-cased/resolve/main/tokenizer_config.json not found in cache or force_download set to True, downloading to /root/.cache/huggingface/transformers/tmpbfurp3vh


Downloading:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

storing https://huggingface.co/bert-base-multilingual-cased/resolve/main/tokenizer_config.json in cache at /root/.cache/huggingface/transformers/f55e7a2ad4f8d0fff2733b3f79777e1e99247f2e4583703e92ce74453af8c235.ec5c189f89475aac7d8cbd243960a0655cfadc3d0474da8ff2ed0bf1699c2a5f
creating metadata file for /root/.cache/huggingface/transformers/f55e7a2ad4f8d0fff2733b3f79777e1e99247f2e4583703e92ce74453af8c235.ec5c189f89475aac7d8cbd243960a0655cfadc3d0474da8ff2ed0bf1699c2a5f
https://huggingface.co/bert-base-multilingual-cased/resolve/main/config.json not found in cache or force_download set to True, downloading to /root/.cache/huggingface/transformers/tmpfozyqqh1


Downloading:   0%|          | 0.00/625 [00:00<?, ?B/s]

storing https://huggingface.co/bert-base-multilingual-cased/resolve/main/config.json in cache at /root/.cache/huggingface/transformers/6c4a5d81a58c9791cdf76a09bce1b5abfb9cf958aebada51200f4515403e5d08.0fe59f3f4f1335dadeb4bce8b8146199d9083512b50d07323c1c319f96df450c
creating metadata file for /root/.cache/huggingface/transformers/6c4a5d81a58c9791cdf76a09bce1b5abfb9cf958aebada51200f4515403e5d08.0fe59f3f4f1335dadeb4bce8b8146199d9083512b50d07323c1c319f96df450c
loading configuration file https://huggingface.co/bert-base-multilingual-cased/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/6c4a5d81a58c9791cdf76a09bce1b5abfb9cf958aebada51200f4515403e5d08.0fe59f3f4f1335dadeb4bce8b8146199d9083512b50d07323c1c319f96df450c
Model config BertConfig {
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "directionality": "bidi",
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "init

...

{'eval_loss': 0.3261491656303406, 'eval_f1': 0.9261083743842364, 'eval_precision': 0.8826291079812206, 'eval_recall': 0.9740932642487047, 'eval_runtime': 3.3247, 'eval_samples_per_second': 85.723, 'eval_steps_per_second': 1.504, 'epoch': 1.85}


***** Running Evaluation *****
  Num examples = 285
  Batch size = 64


{'eval_loss': 0.32052144408226013, 'eval_f1': 0.9306930693069307, 'eval_precision': 0.8909952606635071, 'eval_recall': 0.9740932642487047, 'eval_runtime': 3.3381, 'eval_samples_per_second': 85.378, 'eval_steps_per_second': 1.498, 'epoch': 2.78}




Training completed. Do not forget to share your model on huggingface.co/models =)




{'train_runtime': 40.1229, 'train_samples_per_second': 63.854, 'train_steps_per_second': 4.038, 'train_loss': 0.3862018820680218, 'epoch': 3.0}


In [34]:
eval_metrics = trainer.evaluate()

***** Running Evaluation *****
  Num examples = 285
  Batch size = 64


{'eval_loss': 0.3120799958705902, 'eval_f1': 0.9330024813895782, 'eval_precision': 0.8952380952380953, 'eval_recall': 0.9740932642487047, 'eval_runtime': 3.3276, 'eval_samples_per_second': 85.647, 'eval_steps_per_second': 1.503, 'epoch': 3.0}


In [35]:
trainer.create_model_card()

Dropping the following result as it does not have all the necessary fields:
{'task': {'name': 'Text Classification', 'type': 'text-classification'}, 'metrics': [{'name': 'F1', 'type': 'f1', 'value': 0.9330024813895782}, {'name': 'Precision', 'type': 'precision', 'value': 0.8952380952380953}, {'name': 'Recall', 'type': 'recall', 'value': 0.9740932642487047}]}
